In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pdp
%matplotlib inline

In [ ]:
trn = pd.read_csv("../input/santander-product-recommendation/train_ver2.csv.zip", nrows=200000)

In [ ]:
# for c in trn.columns:
#     print(c)
#     print(trn[c])
#     print("=="*70)

In [ ]:
col_memo = """
variable,description,original,after,mythought
fecha_dato,日付,object,object,日付型に変換すべき
ncodpers,id,int64,int32,idだからint32で十分そう
ind_empleado,顧客情報,object,category,カテゴリカル変数であることが明記されてる
pais_residencia,顧客居住国,object,category,国だしカテゴリカルでは?
sexo,性別,object,category,2クラス
age,年齢,object,category,年齢もカテゴリカル変数なのか...?(年齢にNAがあったのでカテゴリカルに)
fecha_alta,初契約日,object,object,日付に変換すべき
ind_nuevo,新規登録フラグ(6ヵ月),float64,category,フラグだしカテゴリ変数でよいはず
antiguedad,契約期間(月),object,int32,fecha_altaと被ってるのでは?
indrel,顧客等級(1は優先で99は来月解除),float64,category,int8でも良い
ult_fec_cli_1t,1当顧客だった最後の日付,object,object,日付型に変更すべき、nanが多そう
indrel_1mes,月初め基準の顧客等級,float64,category,int8でも良い、ばらけるとは思わない
tiprel_1mes,月初め基準の顧客関係等級,object,category,indrel_1mesと殆ど同じではという認識
indresi,居住地と銀行所在地の国家の一致,object,category,実際にはバイナリになる
indext,外国人指標(銀行の国と顧客の誕生国が同じか否か),object,category,indresiと同じか逆になるだけでは?
conyuemp,配偶者指標(1だと配偶者あり),float64,category,実際はバイナリ、Nan多そう
canal_entrada,顧客が参加に使用したチャネル,object,category,なんもわからん
indfall,生存指標,object,category,口座が動かないはずなので大事そう
tipodom,住所タイプ(1だとプライマリ),float64,category,categoryで良いはず
cod_prov,地名コード,float64,int16,categoryでもよさそう、128を超えてると怖いのでint16に、NAがありそうなのでcategoryに
nomprov,地名,object,category,あんまりにもばらけるようならstringにした方が良いかも
ind_actividad_cliente,顧客アクティビティ(1だとアクティブ),float64,category,int8でも良い
renta,家庭の総収入,float64,float32,そんなに精度がいるとは思わないのでfloat32でいったん
segmento,分類(1だとVIP、2だと個人、3だと大卒),object,category,格納されている値が02-PARTICULARSみたいになってるので、前処理に注意
ind_ahor_fin_ult1,預金,int64,category,これ以降の項目を買うかどうか判定するのが目的
ind_aval_fin_ult1,保証,int64,category,ほぼ0では?
ind_cco_fin_ult1,当座預金,int64,category,ほぼ1になってるみたい
ind_cder_fin_ult1,派生預金,int64,category,ほぼ0
ind_cno_fin_ult1,給与口座,int64,category,ほぼ0
ind_ctju_fin_ult1,子供用口座,int64,category,ほぼ0
ind_ctma_fin_ult1,マス用特殊口座,int64,category,ほぼ0
ind_ctop_fin_ult1,特殊口座,int64,category,ほぼ0
ind_ctpp_fin_ult1,特殊プラス口座,int64,category,ほぼ0
ind_deco_fin_ult1,短期預金,int64,category,ほぼ0
ind_deme_fin_ult1,中期預金,int64,category,ほぼ0
ind_dela_fin_ult1,長期預金,int64,category,ほぼ0
ind_ecue_fin_ult1,オンライン口座,int64,category,ほぼ0
ind_fond_fin_ult1,ファンド,int64,category,ほぼ0
ind_hip_fin_ult1,住宅ローン,int64,category,ほぼ0
ind_plan_fin_ult1,年金,int64,category,ほぼ0
ind_pres_fin_ult1,ローン,int64,category,ほぼ0
ind_reca_fin_ult1,税金,int64,category,ほぼ0
ind_tjcr_fin_ult1,クレジットカード,int64,category,ほぼ0
ind_valo_fin_ult1,証券,int64,category,ほぼ0
ind_viv_fin_ult1,自店売買勘定,int64,category,ほぼ0
ind_nomina_ult1,給与,int64,category,ほぼ0、cnoと一緒では?
ind_nom_pens_ult1,年金,int64,category,planと一緒では?
ind_recibo_ult1,デビットカード,int64,category,ほぼ0
"""

if os.path.exists("./col_memo.csv") is True:
    os.remove("./col_memo.csv")

with open("./col_memo.csv", mode="a") as f:
    f.write(col_memo)

### データが綺麗でなく、NAがintのところに入っていたりするので、intを全てcategoryとして読み込む

In [ ]:
col_memo = pd.read_csv("./col_memo.csv")
# col_memo[col_memo["after"].str.contains("int")]["after"] = "category"
col_memo.loc[col_memo["after"].str.contains("int"), "after"] = "category"
col_memo

## 変数メモを基に再度trnを読み込む

In [ ]:
col_memo.columns

In [ ]:
dic = {}
for index, row in col_memo.iterrows():
    dic[row["variable"]] = row["after"]
dic

In [ ]:
# del(trn)
trn = pd.read_csv("../input/santander-product-recommendation/train_ver2.csv.zip", dtype=dic)
trn
# #262行目にエラーがあるっぽい→ageにNAが入っていたのでカテゴリカルで対応
# # trn = pd.read_csv("../input/santander-product-recommendation/train_ver2.csv.zip",nrows=262)
# # trn.iloc[-1]

In [ ]:
trn.to_pickle("./trn_cateory.pkl.gz", compression="gzip")

### trnはpickleで保存してあるのでそれを読み込む

In [ ]:
trn = pd.read_pickle("../input/k/calucurecalu/santander-product-recommendation/trn_category.pkl.gz")
trn

### fecha_datoとfecha_altaをDatetime型に変換

In [ ]:
trn["fecha_dato"] = pd.to_datetime(trn["fecha_dato"],format="%Y-%m-%d")
trn["fecha_alta"] = pd.to_datetime(trn["fecha_alta"],format="%Y-%m-%d")
trn["ult_fec_cli_1t"] = pd.to_datetime(trn["fecha_alta"],format="%Y-%m-%d")
trn.info()

In [ ]:
# rentaの表示
trn["renta"].head()

### カテゴリカル変数がどんな値をしているのか確認する

In [ ]:
for c in trn.select_dtypes(include="category").columns:
    print(f"{c} : {trn[c].cat.categories}")

In [ ]:
#cod_prov == nomprov?
fig = plt.figure()
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)
sns.countplot(x=trn["cod_prov"], ax=ax1)
sns.countplot(x=trn["nomprov"], ax=ax2)


(trn["cod_prov"].value_counts().to_numpy() == trn["nomprov"].value_counts().to_numpy()).all()


#cod_prov と nomprovは全く同一

In [ ]:
# len(trn.dropna()).agg

In [ ]:
# indrel_1mesを修正

print(trn["indrel_1mes"].unique())

# 処理速度が遅いのでmap()を使うように修正
# trn.loc[trn["indrel_1mes"]=="1.0", "indrel_1mes"] = "1"
# trn.loc[trn["indrel_1mes"]=="2.0", "indrel_1mes"] = "2"
# trn.loc[trn["indrel_1mes"]=="3.0", "indrel_1mes"] = "3"
# trn.loc[trn["indrel_1mes"]=="4.0", "indrel_1mes"] = "4"
# trn.loc[trn["indrel_1mes"]=="P", "indrel_1mes"] = "5"

trn["indrel_1mes"] = trn["indrel_1mes"].map({"1.0":"1", "2.0":"2", "3.0":"3", "4.0":"4", "P":"5"})

print(trn["indrel_1mes"].unique())


In [ ]:
skip_cols = ["ncodpers", "renta"]
trn_vis = trn[trn.columns[:24]].drop(columns=skip_cols)
trn_vis.columns

In [ ]:
#  profile = pdp.ProfileReport(trn_vis, title="santander_recommendation", explorative=True)
# profile.to_widgets()

In [ ]:
# for c in trn_vis.columns:
#     print("-"*50)
#     f,ax = plt.subplots(figsize=(20,15))
#     sns.countplot(x=c, data=trn_vis, alpha=0.5)
#     plt.show()

In [ ]:
trn_target = trn.iloc[:,24:].columns
trn_shoken = trn[["fecha_dato",*trn_target]].groupby("fecha_dato")
# for time, tab in trn_shoken:
#     tab[tab.columns[tab.columns!="fecha_dato"]].astype("int8")
#     # nanが多すぎてエラー

# trn_bar.plot.bar(stacked=True, figsize=(20,15))